# Data Saving and Loading

This file contains the code for saving and loading misaligned tomographic data with misalignment vector labels as numpy n-d arrays.

In [1]:
# Import essential packages
import os
import numpy as np
import matplotlib.pyplot as plt

# Import tomography and imaging packages
import tomopy
from skimage.transform import rotate, AffineTransform
from skimage import transform as tf

!pwd

/home/liam/Projects/Tomographic Alignment


In [2]:
# Resolution of shepp3d
res = 128

# Number of entries, recommend using 20% for testing
entries = 1250

In [3]:
# Define function for artificial misalignment
def misalign(prj, mis_axis, ang_tilt = False, noise = False, background = False):
    num_prj, col, row = prj.shape
    dx = mis_axis[:, 0]
    dy = mis_axis[:, 1]
    prj_tmp = tomopy.shift_images(prj, dx, dy)
    
    for i in range(num_prj):
        d_row, d_col, d_ang = mis_axis[i]
        
        if ang_tilt == True:
            prj_tmp[i, :, :] = rotate(prj[i,:,:], d_ang)
        else:
            prj_tmp[i, :, :] = prj[i,:,:]
        
        if noise == True:
            prj_tmp[i, :, :] = random_noise(prj_tmp[i, :, :], mode = 'gaussian')
            
        if background == True:
            prj_tmp[i, :, :] = prj_tmp[i, :, :]+np.random.random()/5
            prj_tmp[i, :, :] = prj_tmp[i, :, :]/prj_tmp[i, :, :].max()  
            
    return prj_tmp

In [4]:
# Creating ground truth tomography
tomo = tomopy.shepp3d(res)
ang = tomopy.angles(180)
proj = tomopy.project(tomo, ang)

In [ ]:
# Create dataset to store misaligned projections and 
dataset = np.zeros((entries, 2), dtype = object)

for i in range(entries):
    # Randomly determined misalignment axis
    mis_axis = np.random.normal(0, 1, (200, 3))
    mis_axis[:, :1] = mis_axis[:, :1]*4
    mis_axis = np.round(mis_axis).astype(int)
    mis_axis_in = np.expand_dims(mis_axis, axis = 0)
    
    proj_mis = misalign(proj.copy(), mis_axis, ang_tilt = True)
    proj_mis = np.expand_dims(proj_mis, axis = 0)
    proj_mis = np.expand_dims(proj_mis, axis = 0)
    
    dataset[i, 0] = proj_mis
    dataset[i, 1] = np.concatenate((mis_axis_in[:, :180, 0], mis_axis_in[:, :180, 1]), axis = 1)
    
    np.save('./shepp{}-{}/shepp{}-{}_{}'.format(res, entries, res, entries, i), dataset[i, :])

In [ ]:
# Checking shape of dataset
print(dataset.shape)
print(dataset[0].shape)
print(dataset[0][0].shape)
print(dataset[0][1].shape)

In [ ]:
test = np.load('./shepp{}-{}/shepp{}-{}_0.npy'.format(res, entries, res, entries), allow_pickle = True)
print(test.shape)
print(test[0].shape)
print(test[1].shape)

In [ ]:
data = []

for i in range(entries):
    data.append(np.load('./shepp{}-{}/shepp{}-{}_{}'.format(res, entries, res, entries, i), allow_pickle = True))

In [ ]:
# Checking shape of data
data = np.asarray(data) 
print(data.shape)
print(data[0].shape)
print(data[0][0].shape)
print(data[0][1].shape)